Connected to Python 3.10.11

In [6]:

from utils import *
from strategy_functions import *

months = [1]
year = 2022
diff_strike = [0]
dte_min = 14
dte_max = 40
option_type = [1]
time_open = '10:00'
time_close = '17:25'
time_open = pd.to_datetime(time_open, format="%H:%M").time()

files = glob.glob(quotes_path+f'/{year}/*')
files = [file for file in files if datetime.strptime(
    file[-18:-8], '%Y-%m-%d').month in months]

id_options_list = []
l = []
for file in files:

    df = pd.read_parquet(file)

    if id_options_list:
        cond_in_id_list = (df.mispar_hoze.isin(id_options_list))
        cond_time_open = (df['timestamp'].dt.time >= time_open)
        df_open = df.loc[cond_in_id_list & cond_time_open ]
        df_open = get_morning_quotes(df_open)

        l.append(df_open)

    df_ta35 = ta35_df_by_date(df)

    ta35_close = get_ta35_by_time(df_ta35, time_close)

    # Get relevant id's by mimush ta35
    df = filter_relevant_options(df, ta35_close)

    df = filter_close_time(df, time_close)

    df_close = get_night_quotes(df)

    l.append(df_close)

    id_options_list = df_close.mispar_hoze.to_list()


data = pd.concat(l, axis=0, ignore_index=True, sort=False)
data = remove_non_relevant_rows(data).reset_index(drop=True)
data = sell_buy_in_same_row_df(data, short=False)
data['profit'] = data['sell'] - data['buy']
data = add_options_details(data, year)
data = filter_option_type(data,call=False)
data['accum_profit'] = data.profit.cumsum()
data['trade_id'] = range(1, len(data) + 1)

data


,trade_id,mispar_hoze,buy,sell,timestamp_buy,timestamp_sell,profit,option_type,mimush,month,year,accum_profit
0,1,83784462,3160,2670,2022-01-02 15:59:58.280,2022-01-03 10:00:00.030,-490,0,1990,1,2022,-490
2,2,83784462,3000,2680,2022-01-03 17:24:36.960,2022-01-04 10:00:00.630,-320,0,1990,1,2022,-810
4,3,83749622,2880,3130,2022-01-04 17:24:53.160,2022-01-05 10:00:00.320,250,0,2000,1,2022,-560
7,4,83749630,2780,4440,2022-01-05 17:24:55.670,2022-01-06 10:00:00.700,1660,0,2020,1,2022,1100
8,5,83749622,2680,2540,2022-01-06 17:24:59.650,2022-01-09 10:00:00.520,-140,0,2000,1,2022,960
11,6,83784462,3190,2750,2022-01-09 16:00:10.920,2022-01-10 10:00:00.120,-440,0,1990,1,2022,520
12,7,83749614,3060,2150,2022-01-10 17:24:47.860,2022-01-11 10:00:09.010,-910,0,1980,1,2022,-390
15,8,83749630,2580,1980,2022-01-11 17:24:59.250,2022-01-12 10:00:00.400,-600,0,2020,1,2022,-990
16,9,83749655,2630,2800,2022-01-12 17:24:55.380,2022-01-13 10:00:00.310,170,0,2060,1,2022,-820
19,10,83784504,2600,2700,2022-01-13 17:24:59.710,2022-01-16 10:00:03.700,100,0,2070,1,2022,-720


In [ ]:
import pandas as pd 

# Create a sample DataFrame with a 'time' column
data = {'time': ['16:30:00', '17:15:00', '18:00:00', '19:45:00', '20:00:00']}
df = pd.DataFrame(data)

# Convert the 'time' column to datetime objects
df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S')

# Define the target time
target_time = pd.to_datetime('17:25', format='%H:%M')

# Calculate the time difference for each row
df['time_difference'] = abs(df['time'] - target_time)

# Find the row with the smallest time difference (closest time)
closest_row = df.loc[df['time_difference'].idxmin()]

print("Closest time:", closest_row['time'].strftime('%H:%M:%S'))


In [24]:
from strategy_functions import *
from utils import *
class StartegyCommon():
    def __init__(self, months=[1], year=2021, dte_min=30,dte_max=31,
                 diff_strike=[0], option_type=1,):
        self.months = [months] if isinstance(months, int) else months or list(range(1, 13))
        self.year = year
        self.dte_min = dte_min
        self.dte_max = dte_max
        self.diff_strike = diff_strike
        self.diff_strike = diff_strike
        self.option_type = option_type
        
    def get_relevant_files(self):
        year_files = get_quotes_by_years(years=self.year)
        files = [file for file in year_files if datetime.strptime(file[-18:-8], '%Y-%m-%d').month in self.months]

        return files    
class Open_close_strategy(StartegyCommon):
    def __init__(self, time_open='10:00',time_close='17:25'):
        super().__init__()
        self.time_open = pd.to_datetime(time_open, format="%H:%M")       
        self.time_close = pd.to_datetime(time_close, format="%H:%M")   
    def final_result(self, data):
        '''
        Resules of Strategy
        Profit/loss
        '''
        data['profit'] = data['sell'] - data['buy']
        data['accum_profit'] = data.profit.cumsum()
        data['trade_id'] = range(1, len(data) + 1)

        return data
a = Open_close_strategy()
files = a.get_relevant_files()
a.dte_min

30

In [5]:
from strategy_functions import *
from utils import *
class StartegyCommon():
    def __init__(self, months=[1], year=2021, dte_min=30,dte_max=31,
                 diff_strike=[0], option_type=1,):
        self.months = [months] if isinstance(months, int) else months or list(range(1, 13))
        self.year = year
        self.dte_min = dte_min
        self.dte_max = dte_max
        self.diff_strike = diff_strike
        self.diff_strike = diff_strike
        self.option_type = option_type
    
    def get_relevant_files(self):
        year_files = get_quotes_by_years(years=self.year)
        files = [file for file in year_files if datetime.strptime(file[-18:-8], '%Y-%m-%d').month in self.months]
        return files 
    
    def filter_by_dte(self, df):
        cond_dte = (df.dte >= self.dte_min) & (df.dte <= self.dte_max)
        df = df.loc[cond_dte]
        
        if len(df) ==0:
            print(f'No data between {self.dte_min} to {self.dte_max}')
        return df
    
class Open_close_strategy(StartegyCommon):
    def __init__(self, time_open='10:00',time_close='17:25'):
        super().__init__()
        self.time_open = pd.to_datetime(time_open, format="%H:%M")       
        self.time_close = pd.to_datetime(time_close, format="%H:%M")   
        
    def proccess_data(self):
        '''
        extracting the relevant data from user's input
        trade in each row with buy/sell  
        '''
        day_before_options_id = None
        l = []  
        # files by months
        files = self.get_relevant_files()
        
        for file in files:
            df = pd.read_parquet(file)
            if day_before_options_id: # When there are options from night before
                cond_in_id_list = (df.mispar_hoze.isin(day_before_options_id))
                cond_time_open = (df['timestamp'].dt.time >= self.time_open.time())
                # filter by conditions
                df_open = df.loc[cond_in_id_list & cond_time_open ]
                df_open = get_morning_quotes(df_open)
                l.append(df_open)    
            df_ta35 = ta35_df_by_date(df)
            ta35_close = get_ta35_by_time(df_ta35, self.time_close)
            # Get relevant id's by mimush ta35
            df = self.filter_by_dte()
            df = filter_close_time(df, self.time_close)
            df_close = get_night_quotes(df)
            l.append(df_close)
            day_before_options_id = df_close.mispar_hoze.to_list()
        data = pd.concat(l, axis=0, ignore_index=True, sort=False)
        # Remove rows where no trades for example option at the end of day  without open day after that
        data = remove_non_trade_rows(data).reset_index(drop=True)
        # Arrange the data whrer sell and buy will be in the same row, easier to compute (sum of columns)
        data = sell_buy_in_same_row_df(data, short=False)
        # Add option details such Call/Put, date to exipration
        data = add_options_details(data, self.year)
        # Filter by Call/Put data only
        data = filter_option_type(data,call=False)
        # Filter by days to expiration
        data = filter_by_dte(data,dte_min=self.dte_min,dte_max =self.dte_max)
        return data
    
    def final_result(self, data):
        '''
        Resules of Strategy
        Profit/loss
        '''
        data['profit'] = data['sell'] - data['buy']
        data['accum_profit'] = data.profit.cumsum()
        data['trade_id'] = range(1, len(data) + 1)

        return data
    
    
a = Open_close_strategy()
files = a.get_relevant_files()
a.dte_min

30

None


In [2]:
day_before_options_id = None
l = []  
for file in files[1:3]:
    df = pd.read_parquet(file)
    if day_before_options_id: # When there are options from night before
        cond_in_id_list = (df.mispar_hoze.isin(day_before_options_id))
        cond_time_open = (df['timestamp'].dt.time >= a.time_open.time())
        # Filter by conditions
        df_open = df.loc[cond_in_id_list & cond_time_open]
        # Get first quotes 
        df_open = get_morning_quotes(df_open)
        l.append(df_open)
    # Get ta35 index
    ta35_close = get_ta35_by_time(df, a.time_close)


In [3]:
df = a.filter_by_dte(df)

No data between 30 to 31


,mispar_hoze,p1_Bid,p2_Bid,q1_Bid,q2_Bid,p1_Ask,p2_Ask,q1_Ask,q2_Ask,ta35,timestamp,option_type,mimush,month,year,dte,diff_strike


In [ ]:
    # Get relevant id's by mimush from ta35 index and days to exipration
    df = filter_relevant_options(df, ta35_close)
    df = filter_close_time(df, a.time_close)
    df_close = get_night_quotes(df)
    l.append(df_close)
    day_before_options_id = df_close.mispar_hoze.to_list()
data = pd.concat(l, axis=0, ignore_index=True, sort=False)


In [33]:
data = remove_non_trade_rows(data).reset_index(drop=True)
data

,mispar_hoze,p1_Bid,p1_Ask,timestamp,open
0,83783704,3380,3450,2022-01-03 17:24:59.520,0
1,83783704,3650,3700,2022-01-04 10:00:16.060,1
2,83784462,2950,3000,2022-01-03 17:24:36.960,0
3,83784462,2680,2710,2022-01-04 10:00:00.630,1


In [5]:
long = sell_buy_in_same_row_df(data, short=False)
long

,trade_id,mispar_hoze,buy,sell,timestamp_close,timestamp_open
0,1,83380667,2260,1880,2021-01-04 17:24:48.320,2021-01-05 10:00:15.140
1,2,83381251,3480,4020,2021-01-04 17:24:57.790,2021-01-05 10:00:36.960


In [7]:
short = sell_buy_in_same_row_df(data, short=True)
short

,trade_id,mispar_hoze,sell,buy,timestamp_close,timestamp_open
0,1,83380667,2220,1910,2021-01-04 17:24:48.320,2021-01-05 10:00:15.140
1,2,83381251,3440,4120,2021-01-04 17:24:57.790,2021-01-05 10:00:36.960


In [8]:
short = add_options_details(short, a.year)
# short = filter_option_type(short,call=False)

short

,trade_id,mispar_hoze,sell,buy,timestamp_close,timestamp_open,option_type,mimush,month,year,pkiya
0,1,83380667,2220,1910,2021-01-04 17:24:48.320,2021-01-05 10:00:15.140,1,1510,1,2021,2021-01-28
1,2,83381251,3440,4120,2021-01-04 17:24:57.790,2021-01-05 10:00:36.960,0,1510,1,2021,2021-01-28


In [10]:
short = filter_option_type(short,call=False)
short

,trade_id,mispar_hoze,sell,buy,timestamp_close,timestamp_open,option_type,mimush,month,year,pkiya
1,2,83381251,3440,4120,2021-01-04 17:24:57.790,2021-01-05 10:00:36.960,0,1510,1,2021,2021-01-28


In [20]:
a.final_result(short)

short = filter_by_dte(short, a.dte_min, a.dte_max)

,trade_id,mispar_hoze,sell,buy,timestamp_buy,timestamp_sell,option_type,mimush,month,year,pkiya,profit,accum_profit
1,1,83381251,3440,4120,2021-01-04 17:24:57.790,2021-01-05 10:00:36.960,0,1510,1,2021,2021-01-28,-680,-680


In [28]:
if not 'year' in short.columns

,trade_id,mispar_hoze,sell,buy,timestamp_buy,timestamp_sell,option_type,mimush,month,year,pkiya
1,2,83381251,3440,4120,2021-01-04 17:24:57.790,2021-01-05 10:00:36.960,0,1510,1,2021,2021-01-28


In [38]:
'dte_min' not in short.columns or 'dte_max' not in short.columns

True

In [20]:
short['pkiya'] = pd.to_datetime(short['pkiya'], format="%Y-%m-%d")
short.pkiya

1   2021-01-28
Name: pkiya, dtype: datetime64[ns]

In [21]:
# Convert 'date' variable to datetime in order to substruct
date_ = short.timestamp_close.dt.normalize()
date_



1   2021-01-04
Name: timestamp_close, dtype: datetime64[ns]

In [ ]:
a.final_result(short)

,mispar_hoze,p1_Bid,p2_Bid,q1_Bid,q2_Bid,p1_Ask,p2_Ask,q1_Ask,q2_Ask,ta35,timestamp,option_type,mimush,month,year,dte,diff_strike,open
0,83380667,2220,2210,4,1,2260,2350,6,10,1500,2021-01-04 17:24:48.320,1,1510,1,2021,24,10,0
1,83381251,3440,3430,1,3,3480,3490,2,3,1500,2021-01-04 17:24:57.790,0,1510,1,2021,24,10,0
2,83380667,1880,1870,15,1,1910,1920,1,1,1490,2021-01-05 10:00:15.140,1,1510,1,2021,23,20,1
3,83381251,4020,4010,3,1,4120,4130,4,1,1490,2021-01-05 10:00:36.960,0,1510,1,2021,23,20,1
4,83339895,2650,2640,1,7,2680,2690,1,1,1500,2021-01-05 17:24:58.740,1,1500,1,2021,23,0,0
5,83340422,2880,2870,6,3,2920,2930,1,1,1500,2021-01-05 17:24:59.330,0,1500,1,2021,23,0,0


In [22]:
# Substruct expiration date from date input
short['dte'] = (short.pkiya - date_).dt.days
short

,trade_id,mispar_hoze,sell,buy,timestamp_close,timestamp_open,option_type,mimush,month,year,pkiya,dte
1,2,83381251,3440,4120,2021-01-04 17:24:57.790,2021-01-05 10:00:36.960,0,1510,1,2021,2021-01-28,24
